In [351]:
%pip install python-metar

Note: you may need to restart the kernel to use updated packages.


In [352]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from metar import Metar
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [353]:
bimtra_df = pd.read_csv('data/bimtra_df.csv')
esperas_df = pd.read_csv('data/esperas_df.csv')
metaf_df = pd.read_csv('data/metaf_df.csv')
metar_df = pd.read_csv('data/metar_df.csv')
tc_prev_df = pd.read_csv('data/tc-prev_df.csv')
tc_real_df = pd.read_csv('data/tc-real_df.csv')
test_data = pd.read_csv('data/idsc_test.csv')

In [354]:
bimtra_df.dt_arr = bimtra_df.dt_arr.apply(lambda x: datetime.datetime.utcfromtimestamp(int(x)/1000))
bimtra_df.dt_dep = bimtra_df.dt_dep.apply(lambda x: datetime.datetime.utcfromtimestamp(int(x)/1000))
esperas_df.hora = esperas_df.hora.apply(lambda x: datetime.datetime.utcfromtimestamp(int(x)/1000))
metaf_df.hora = metaf_df.hora.apply(lambda x: datetime.datetime.utcfromtimestamp(int(x)/1000))
metar_df.hora = metar_df.hora.apply(lambda x: datetime.datetime.utcfromtimestamp(int(x)/1000))
tc_prev_df.hora = tc_prev_df.hora.apply(lambda x: datetime.datetime.utcfromtimestamp(int(x)/1000))
tc_real_df.hora = tc_real_df.hora.apply(lambda x: datetime.datetime.utcfromtimestamp(int(x)/1000))
test_data.hora_ref = pd.to_datetime(test_data.hora_ref)
test_data.dt_dep = pd.to_datetime(test_data.dt_dep)

In [355]:
test_data.head()

,flightid,origem,destino,dt_dep,hora_ref,snapshot_radar,path,hora_esperas,esperas,aero_esperas,...,metaf,aero_metaf,hora_metar,metar,aero_metar,hora_tcp,troca,aero_tcp,hora_tcr,aero_tcr
0,7945735584a3297121c4f5ae0de8ecd1,SBKP,SBSV,2023-05-30 11:36:03,2023-05-30 11:00:00,MULTIPOINT ((-0.8807200448127578 -0.5148453155...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-30 10:00:00.000,0,SBSV,...,NaN,NaN,2023-05-30 11:00:00.000,METAR SBSV 301100Z 12009KT 9999 FEW023 28/21 Q...,SBSV,2023-05-30 12:00:00.000,0,SBSV,NaN,NaN
1,85cb958e8b9d583575cd9cb93e389226,SBFL,SBRJ,2023-05-19 08:26:06,2023-05-19 08:00:00,MULTIPOINT ((-0.8826266039383116 -0.4926305156...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-19 07:00:00.000,0,SBRJ,...,METAF SBRJ 190900Z 33004KT 9999 BKN033 ...,SBRJ,2023-05-19 08:00:00.000,METAR SBRJ 190800Z 02004KT 9999 FEW010 SCT040 ...,SBRJ,2023-05-19 09:00:00.000,0,SBRJ,NaN,NaN
2,2d4e8cd3947525a12ff16af520beefb9,SBCF,SBGR,2023-05-22 11:52:02,2023-05-22 11:00:00,MULTIPOINT ((-0.8972512464147154 -0.5215196550...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-22 10:00:00.000,0,SBGR,...,METAF SBGR 221200Z 04003KT CAVOK 19/09 Q1021=\n,SBGR,2023-05-22 11:00:00.000,METAR SBGR 221100Z 07006KT CAVOK 13/12 Q1022=,SBGR,2023-05-22 12:00:00.000,0,SBGR,NaN,NaN
3,24cf317a1494a016a018d9f5cd183639,SBFL,SBSP,2023-05-29 16:07:07,2023-05-29 16:00:00,MULTIPOINT ((-0.892972783728755 -0.51704889771...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-29 15:00:00.000,2,SBSP,...,METAF SBSP 291700Z 15010KT 9999 OVC033 ...,SBSP,2023-05-29 16:00:00.000,METAR SBSP 291600Z 12010KT 090V160 9999 SCT015...,SBSP,2023-05-29 17:00:00.000,0,SBSP,NaN,NaN
4,370babaf32dc82a6a64bf5b71a3434ed,SBCF,SBKP,2023-05-25 22:16:04,2023-05-25 22:00:00,MULTIPOINT ((-0.8970045369724385 -0.5240544927...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-25 21:00:00.000,0,SBKP,...,METAF SBKP 252300Z 08008KT CAVOK 21/12 Q1019=\n,SBKP,2023-05-25 22:00:00.000,METAR SBKP 252200Z 09004KT CAVOK 23/13 Q1020=,SBKP,2023-05-25 23:00:00.000,0,SBKP,NaN,NaN


In [356]:
test_data.columns

Index(['flightid', 'origem', 'destino', 'dt_dep', 'hora_ref', 'snapshot_radar',
       'path', 'hora_esperas', 'esperas', 'aero_esperas', 'hora_metaf',
       'metaf', 'aero_metaf', 'hora_metar', 'metar', 'aero_metar', 'hora_tcp',
       'troca', 'aero_tcp', 'hora_tcr', 'aero_tcr'],
      dtype='object')

In [357]:
bimtra_df['solution'] = ((bimtra_df.dt_arr - bimtra_df.dt_dep).dt.total_seconds()).astype(int)
bimtra_df.head()

,flightid,origem,destino,dt_dep,dt_arr,solution
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-06-01 00:44:57,2022-06-01 01:26:00,2463
1,c7c5c10716335b048f86d8c52fcba3f2,SBGR,SBRJ,2022-06-01 00:57:01,2022-06-01 01:32:53,2152
2,3162de7203a972f071d5a48e8f0f4828,SBBR,SBRJ,2022-06-01 00:26:52,2022-06-01 01:43:45,4613
3,8fcd243e9b9eebbe62ab145ce04b6ab5,SBKP,SBCF,2022-06-01 00:58:40,2022-06-01 01:41:41,2581
4,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,2022-06-01 01:00:38,2022-06-01 02:33:05,5547


In [358]:
display(test_data.head())

display(bimtra_df.head())
display(esperas_df.head())
# display(f'metaf_df: {metaf_df.columns}')
# display(f'metar_df: {metar_df.columns}')
display(tc_prev_df.head())
display(tc_real_df.head())

,flightid,origem,destino,dt_dep,hora_ref,snapshot_radar,path,hora_esperas,esperas,aero_esperas,...,metaf,aero_metaf,hora_metar,metar,aero_metar,hora_tcp,troca,aero_tcp,hora_tcr,aero_tcr
0,7945735584a3297121c4f5ae0de8ecd1,SBKP,SBSV,2023-05-30 11:36:03,2023-05-30 11:00:00,MULTIPOINT ((-0.8807200448127578 -0.5148453155...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-30 10:00:00.000,0,SBSV,...,NaN,NaN,2023-05-30 11:00:00.000,METAR SBSV 301100Z 12009KT 9999 FEW023 28/21 Q...,SBSV,2023-05-30 12:00:00.000,0,SBSV,NaN,NaN
1,85cb958e8b9d583575cd9cb93e389226,SBFL,SBRJ,2023-05-19 08:26:06,2023-05-19 08:00:00,MULTIPOINT ((-0.8826266039383116 -0.4926305156...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-19 07:00:00.000,0,SBRJ,...,METAF SBRJ 190900Z 33004KT 9999 BKN033 ...,SBRJ,2023-05-19 08:00:00.000,METAR SBRJ 190800Z 02004KT 9999 FEW010 SCT040 ...,SBRJ,2023-05-19 09:00:00.000,0,SBRJ,NaN,NaN
2,2d4e8cd3947525a12ff16af520beefb9,SBCF,SBGR,2023-05-22 11:52:02,2023-05-22 11:00:00,MULTIPOINT ((-0.8972512464147154 -0.5215196550...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-22 10:00:00.000,0,SBGR,...,METAF SBGR 221200Z 04003KT CAVOK 19/09 Q1021=\n,SBGR,2023-05-22 11:00:00.000,METAR SBGR 221100Z 07006KT CAVOK 13/12 Q1022=,SBGR,2023-05-22 12:00:00.000,0,SBGR,NaN,NaN
3,24cf317a1494a016a018d9f5cd183639,SBFL,SBSP,2023-05-29 16:07:07,2023-05-29 16:00:00,MULTIPOINT ((-0.892972783728755 -0.51704889771...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-29 15:00:00.000,2,SBSP,...,METAF SBSP 291700Z 15010KT 9999 OVC033 ...,SBSP,2023-05-29 16:00:00.000,METAR SBSP 291600Z 12010KT 090V160 9999 SCT015...,SBSP,2023-05-29 17:00:00.000,0,SBSP,NaN,NaN
4,370babaf32dc82a6a64bf5b71a3434ed,SBCF,SBKP,2023-05-25 22:16:04,2023-05-25 22:00:00,MULTIPOINT ((-0.8970045369724385 -0.5240544927...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-25 21:00:00.000,0,SBKP,...,METAF SBKP 252300Z 08008KT CAVOK 21/12 Q1019=\n,SBKP,2023-05-25 22:00:00.000,METAR SBKP 252200Z 09004KT CAVOK 23/13 Q1020=,SBKP,2023-05-25 23:00:00.000,0,SBKP,NaN,NaN


,flightid,origem,destino,dt_dep,dt_arr,solution
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-06-01 00:44:57,2022-06-01 01:26:00,2463
1,c7c5c10716335b048f86d8c52fcba3f2,SBGR,SBRJ,2022-06-01 00:57:01,2022-06-01 01:32:53,2152
2,3162de7203a972f071d5a48e8f0f4828,SBBR,SBRJ,2022-06-01 00:26:52,2022-06-01 01:43:45,4613
3,8fcd243e9b9eebbe62ab145ce04b6ab5,SBKP,SBCF,2022-06-01 00:58:40,2022-06-01 01:41:41,2581
4,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,2022-06-01 01:00:38,2022-06-01 02:33:05,5547


,esperas,hora,aero
0,0,2022-06-01 00:00:00,SBBR
1,0,2022-06-01 01:00:00,SBBR
2,0,2022-06-01 02:00:00,SBBR
3,0,2022-06-01 03:00:00,SBBR
4,0,2022-06-01 04:00:00,SBBR


,hora,troca,aero
0,2022-06-01 00:00:00,0,BR
1,2022-06-01 01:00:00,0,BR
2,2022-06-01 02:00:00,0,BR
3,2022-06-01 03:00:00,0,BR
4,2022-06-01 04:00:00,0,BR


,hora,nova_cabeceira,antiga_cabeceira,aero
0,2022-06-01 14:14:03,32,03,FL
1,2022-06-01 18:51:10,18,12,RF
2,2022-06-01 19:12:25,16,13,CF
3,2022-06-01 19:12:38,15,26,GL
4,2022-06-01 19:12:57,15,26,GL


In [359]:
bimtra_df['hora_ref'] = bimtra_df.dt_dep.apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
bimtra_df.head()

,flightid,origem,destino,dt_dep,dt_arr,solution,hora_ref
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-06-01 00:44:57,2022-06-01 01:26:00,2463,2022-06-01 00:00:00
1,c7c5c10716335b048f86d8c52fcba3f2,SBGR,SBRJ,2022-06-01 00:57:01,2022-06-01 01:32:53,2152,2022-06-01 00:00:00
2,3162de7203a972f071d5a48e8f0f4828,SBBR,SBRJ,2022-06-01 00:26:52,2022-06-01 01:43:45,4613,2022-06-01 00:00:00
3,8fcd243e9b9eebbe62ab145ce04b6ab5,SBKP,SBCF,2022-06-01 00:58:40,2022-06-01 01:41:41,2581,2022-06-01 00:00:00
4,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,2022-06-01 01:00:38,2022-06-01 02:33:05,5547,2022-06-01 01:00:00


In [360]:
display(f'idsc_test_df: {test_data.columns}')

display(f'bimtra_df: {bimtra_df.columns}')
display(f'esperas_df: {esperas_df.columns}')
# display(f'metaf_df: {metaf_df.columns}')
# display(f'metar_df: {metar_df.columns}')
display(f'tc_prev_df: {tc_prev_df.columns}')
display(f'tc_real_df: {tc_real_df.columns}')

"idsc_test_df: Index(['flightid', 'origem', 'destino', 'dt_dep', 'hora_ref', 'snapshot_radar',\n       'path', 'hora_esperas', 'esperas', 'aero_esperas', 'hora_metaf',\n       'metaf', 'aero_metaf', 'hora_metar', 'metar', 'aero_metar', 'hora_tcp',\n       'troca', 'aero_tcp', 'hora_tcr', 'aero_tcr'],\n      dtype='object')"

"bimtra_df: Index(['flightid', 'origem', 'destino', 'dt_dep', 'dt_arr', 'solution',\n       'hora_ref'],\n      dtype='object')"

"esperas_df: Index(['esperas', 'hora', 'aero'], dtype='object')"

"tc_prev_df: Index(['hora', 'troca', 'aero'], dtype='object')"

"tc_real_df: Index(['hora', 'nova_cabeceira', 'antiga_cabeceira', 'aero'], dtype='object')"

In [361]:
print(f'bimtra_df: {bimtra_df.shape}')
print(f'esperas_df: {esperas_df.shape}')
print(f'metaf_df: {metaf_df.shape}')
print(f'metar_df: {metar_df.shape}')
print(f'tc_prev_df: {tc_prev_df.shape}')
print(f'tc_real_df: {tc_real_df.shape}')

bimtra_df: (300347, 7)
esperas_df: (99936, 3)
metaf_df: (43938, 3)
metar_df: (106744, 3)
tc_prev_df: (99936, 3)
tc_real_df: (33431, 4)


In [362]:
tc_prev_df['aero_tcp'] = tc_prev_df['aero']
tc_prev_df = tc_prev_df.drop(['aero'], axis=1)
tc_real_df['aero_tcr'] = tc_real_df['aero']
tc_real_df = tc_real_df.drop(['aero'], axis=1)

display(tc_prev_df.head())
display(tc_real_df.head())
display(tc_real_df.shape)

,hora,troca,aero_tcp
0,2022-06-01 00:00:00,0,BR
1,2022-06-01 01:00:00,0,BR
2,2022-06-01 02:00:00,0,BR
3,2022-06-01 03:00:00,0,BR
4,2022-06-01 04:00:00,0,BR


,hora,nova_cabeceira,antiga_cabeceira,aero_tcr
0,2022-06-01 14:14:03,32,03,FL
1,2022-06-01 18:51:10,18,12,RF
2,2022-06-01 19:12:25,16,13,CF
3,2022-06-01 19:12:38,15,26,GL
4,2022-06-01 19:12:57,15,26,GL


(33431, 4)

In [363]:
tc_real_df.isna().sum()

hora                0
nova_cabeceira      0
antiga_cabeceira    0
aero_tcr            0
dtype: int64

In [364]:
esperas_df['aero_esperas'] = esperas_df['aero']
esperas_df = esperas_df.drop(['aero'], axis=1)
esperas_df['hora_esperas'] = esperas_df['hora']
esperas_df = esperas_df.drop(['hora'], axis=1)
esperas_df['hora_ref'] = esperas_df['hora_esperas']
esperas_df['destino'] = esperas_df['aero_esperas']

esperas_df.head()

,esperas,aero_esperas,hora_esperas,hora_ref,destino
0,0,SBBR,2022-06-01 00:00:00,2022-06-01 00:00:00,SBBR
1,0,SBBR,2022-06-01 01:00:00,2022-06-01 01:00:00,SBBR
2,0,SBBR,2022-06-01 02:00:00,2022-06-01 02:00:00,SBBR
3,0,SBBR,2022-06-01 03:00:00,2022-06-01 03:00:00,SBBR
4,0,SBBR,2022-06-01 04:00:00,2022-06-01 04:00:00,SBBR


In [365]:
print(f'bimtra_df: {bimtra_df.shape}')
print(f'esperas_df: {esperas_df.shape}')
print(f'metaf_df: {metaf_df.shape}')
print(f'metar_df: {metar_df.shape}')
print(f'tc_prev_df: {tc_prev_df.shape}')
print(f'tc_real_df: {tc_real_df.shape}')

bimtra_df: (300347, 7)
esperas_df: (99936, 5)
metaf_df: (43938, 3)
metar_df: (106744, 3)
tc_prev_df: (99936, 3)
tc_real_df: (33431, 4)


In [366]:
tc_real_df['hora_ref'] = tc_real_df.hora.apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
tc_prev_df['hora_ref'] = tc_prev_df.hora

tc_real_df['hora_tcr'] = tc_real_df.hora
tc_prev_df['hora_tcp'] = tc_prev_df.hora
tc_real_df = tc_real_df.drop(['hora'], axis=1)
tc_prev_df = tc_prev_df.drop(['hora'], axis=1)

tc_real_df['destino'] = tc_real_df.aero_tcr
tc_prev_df['destino'] = tc_prev_df.aero_tcp


display(tc_real_df.head())
display(tc_prev_df.head())

,nova_cabeceira,antiga_cabeceira,aero_tcr,hora_ref,hora_tcr,destino
0,32,03,FL,2022-06-01 14:00:00,2022-06-01 14:14:03,FL
1,18,12,RF,2022-06-01 18:00:00,2022-06-01 18:51:10,RF
2,16,13,CF,2022-06-01 19:00:00,2022-06-01 19:12:25,CF
3,15,26,GL,2022-06-01 19:00:00,2022-06-01 19:12:38,GL
4,15,26,GL,2022-06-01 19:00:00,2022-06-01 19:12:57,GL


,troca,aero_tcp,hora_ref,hora_tcp,destino
0,0,BR,2022-06-01 00:00:00,2022-06-01 00:00:00,BR
1,0,BR,2022-06-01 01:00:00,2022-06-01 01:00:00,BR
2,0,BR,2022-06-01 02:00:00,2022-06-01 02:00:00,BR
3,0,BR,2022-06-01 03:00:00,2022-06-01 03:00:00,BR
4,0,BR,2022-06-01 04:00:00,2022-06-01 04:00:00,BR


In [367]:
tc_prev_df['destino'] = 'SB' + tc_prev_df['destino'].astype(str)
tc_prev_df['destino'].unique()

tc_real_df['destino'] = 'SB' + tc_real_df['destino'].astype(str)
tc_real_df['destino'].unique()

tc_prev_df['aero_tcp'] = 'SB' + tc_prev_df['aero_tcp'].astype(str)
tc_prev_df['aero_tcp'].unique()

tc_real_df['aero_tcr'] = 'SB' + tc_real_df['aero_tcr'].astype(str)
tc_real_df['aero_tcr'].unique()

array(['SBFL', 'SBRF', 'SBCF', 'SBGL', 'SBSV', 'SBKP', 'SBGR', 'SBPA',
       'SBRJ', 'SBSP', 'SBBR', 'SBCT'], dtype=object)

In [368]:
train_data = pd.merge(bimtra_df, esperas_df, how='left', on=['hora_ref', 'destino'])
train_data = pd.merge(train_data, tc_prev_df, how='left', on=['hora_ref', 'destino'])
train_data = pd.merge(train_data, tc_real_df, how='left', on=['hora_ref', 'destino'])
# train_data['hora_ref'] = train_data['hora_ref'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [369]:
display(train_data.sample(10))
display(train_data.shape)


,flightid,origem,destino,dt_dep,dt_arr,solution,hora_ref,esperas,aero_esperas,hora_esperas,troca,aero_tcp,hora_tcp,nova_cabeceira,antiga_cabeceira,aero_tcr,hora_tcr
141382,5d28d5873f113ce8051362c3c9e1dad9,SBGR,SBCF,2022-11-05 19:17:43,2022-11-05 20:04:53,2830,2022-11-05 19:00:00,0,SBCF,2022-11-05 19:00:00,0,SBCF,2022-11-05 19:00:00,NaN,NaN,NaN,NaT
74009,4f36bed2e00035bb127c71bcbf75de48,SBSP,SBCF,2022-08-22 09:24:53,2022-08-22 10:16:29,3096,2022-08-22 09:00:00,0,SBCF,2022-08-22 09:00:00,0,SBCF,2022-08-22 09:00:00,NaN,NaN,NaN,NaT
52227,f7031aace3607ec1571ae8d5813ae94b,SBSP,SBRJ,2022-07-29 18:17:35,2022-07-29 18:56:05,2310,2022-07-29 18:00:00,0,SBRJ,2022-07-29 18:00:00,1,SBRJ,2022-07-29 18:00:00,NaN,NaN,NaN,NaT
40658,da6ed0e23bca4a4fc2fe78c183e1960e,SBKP,SBSV,2022-07-17 20:35:51,2022-07-17 22:35:29,7178,2022-07-17 20:00:00,0,SBSV,2022-07-17 20:00:00,0,SBSV,2022-07-17 20:00:00,17,10,SBSV,2022-07-17 20:15:27
70274,d16fe77dc5d7d369a3a95623d410071d,SBRJ,SBSP,2022-08-18 18:25:58,2022-08-18 19:13:10,2832,2022-08-18 18:00:00,0,SBSP,2022-08-18 18:00:00,1,SBSP,2022-08-18 18:00:00,35L,34,SBSP,2022-08-18 18:44:09
294116,bbc417f9863290ec9bf28ce642de1b79,SBRJ,SBCT,2023-04-06 20:27:53,2023-04-06 21:39:33,4300,2023-04-06 20:00:00,0,SBCT,2023-04-06 20:00:00,0,SBCT,2023-04-06 20:00:00,NaN,NaN,NaN,NaT
322199,c110723b9cfdb6b417c375da96cc5a1d,SBRJ,SBSV,2023-04-25 12:04:28,2023-04-25 13:45:45,6077,2023-04-25 12:00:00,0,SBSV,2023-04-25 12:00:00,0,SBSV,2023-04-25 12:00:00,10,17,SBSV,2023-04-25 12:08:28
171016,6b1eee54b0c042f9fb26fc4c76ac1139,SBSV,SBSP,2022-12-06 12:33:42,2022-12-06 14:52:13,8311,2022-12-06 12:00:00,0,SBSP,2022-12-06 12:00:00,1,SBSP,2022-12-06 12:00:00,NaN,NaN,NaN,NaT
210385,4723774a8e629c831a50210a7ae14bf9,SBSP,SBFL,2023-01-15 18:50:10,2023-01-15 19:40:01,2991,2023-01-15 18:00:00,0,SBFL,2023-01-15 18:00:00,1,SBFL,2023-01-15 18:00:00,03,25,SBFL,2023-01-15 18:18:30
223648,d1d843a5a2072da43fa136db30b3c1ad,SBSP,SBRJ,2023-01-28 21:07:20,2023-01-28 21:55:36,2896,2023-01-28 21:00:00,0,SBRJ,2023-01-28 21:00:00,0,SBRJ,2023-01-28 21:00:00,NaN,NaN,NaN,NaT


(340098, 17)

In [370]:
train_data.isna().sum() 

flightid                 0
origem                   0
destino                  0
dt_dep                   0
dt_arr                   0
solution                 0
hora_ref                 0
esperas                  0
aero_esperas             0
hora_esperas             0
troca                    0
aero_tcp                 0
hora_tcp                 0
nova_cabeceira      230141
antiga_cabeceira    230141
aero_tcr            230141
hora_tcr            230141
dtype: int64

In [371]:
train_data.shape

(340098, 17)

In [372]:
train_data = train_data.drop(columns=['nova_cabeceira', 'antiga_cabeceira', 'dt_arr', 'hora_esperas', 'hora_tcp', 'hora_tcr', 'hora_ref'], axis=1)

train_data = train_data[['flightid', 'origem', 'destino', 'esperas', 'aero_esperas', 'troca', 'aero_tcp', 'aero_tcr', 'solution', 'dt_dep']]

train_data.head()

,flightid,origem,destino,esperas,aero_esperas,troca,aero_tcp,aero_tcr,solution,dt_dep
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,0,SBRJ,0,SBRJ,NaN,2463,2022-06-01 00:44:57
1,c7c5c10716335b048f86d8c52fcba3f2,SBGR,SBRJ,0,SBRJ,0,SBRJ,NaN,2152,2022-06-01 00:57:01
2,3162de7203a972f071d5a48e8f0f4828,SBBR,SBRJ,0,SBRJ,0,SBRJ,NaN,4613,2022-06-01 00:26:52
3,8fcd243e9b9eebbe62ab145ce04b6ab5,SBKP,SBCF,0,SBCF,0,SBCF,NaN,2581,2022-06-01 00:58:40
4,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,0,SBFL,0,SBFL,NaN,5547,2022-06-01 01:00:38


In [373]:
test_data = test_data.drop(columns=['snapshot_radar', 'path', 'hora_metaf', 'metaf', 'aero_metaf', 'hora_metar', 'metar', 'aero_metar', 'hora_esperas', 'hora_tcp', 'hora_tcr', 'hora_ref'], axis=1)
test_data = test_data[['flightid', 'origem', 'destino', 'esperas', 'aero_esperas', 'troca', 'aero_tcp', 'aero_tcr', 'dt_dep']]
test_data.head()

,flightid,origem,destino,esperas,aero_esperas,troca,aero_tcp,aero_tcr,dt_dep
0,7945735584a3297121c4f5ae0de8ecd1,SBKP,SBSV,0,SBSV,0,SBSV,NaN,2023-05-30 11:36:03
1,85cb958e8b9d583575cd9cb93e389226,SBFL,SBRJ,0,SBRJ,0,SBRJ,NaN,2023-05-19 08:26:06
2,2d4e8cd3947525a12ff16af520beefb9,SBCF,SBGR,0,SBGR,0,SBGR,NaN,2023-05-22 11:52:02
3,24cf317a1494a016a018d9f5cd183639,SBFL,SBSP,2,SBSP,0,SBSP,NaN,2023-05-29 16:07:07
4,370babaf32dc82a6a64bf5b71a3434ed,SBCF,SBKP,0,SBKP,0,SBKP,NaN,2023-05-25 22:16:04


In [374]:
cat_columns = ['origem', 'destino', 'aero_esperas', 'aero_tcp', 'aero_tcr']
train_data, test_data = pd.get_dummies(train_data, columns=cat_columns, dtype=int), pd.get_dummies(test_data, columns=cat_columns, dtype=int)

for column in train_data.columns:
    if column == 'solution':
        continue
    if column not in test_data.columns:
        test_data[column] = 0

In [375]:
train_data.head()

,flightid,esperas,troca,solution,dt_dep,origem_SBBR,origem_SBCF,origem_SBCT,origem_SBFL,origem_SBGL,...,aero_tcr_SBCT,aero_tcr_SBFL,aero_tcr_SBGL,aero_tcr_SBGR,aero_tcr_SBKP,aero_tcr_SBPA,aero_tcr_SBRF,aero_tcr_SBRJ,aero_tcr_SBSP,aero_tcr_SBSV
0,fcb2bf90345705318213ae1307c0f901,0,0,2463,2022-06-01 00:44:57,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,c7c5c10716335b048f86d8c52fcba3f2,0,0,2152,2022-06-01 00:57:01,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3162de7203a972f071d5a48e8f0f4828,0,0,4613,2022-06-01 00:26:52,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8fcd243e9b9eebbe62ab145ce04b6ab5,0,0,2581,2022-06-01 00:58:40,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,504a62621cd231d6ab67e674ce538cd3,0,0,5547,2022-06-01 01:00:38,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [376]:
train_data.columns


Index(['flightid', 'esperas', 'troca', 'solution', 'dt_dep', 'origem_SBBR',
       'origem_SBCF', 'origem_SBCT', 'origem_SBFL', 'origem_SBGL',
       'origem_SBGR', 'origem_SBKP', 'origem_SBPA', 'origem_SBRF',
       'origem_SBRJ', 'origem_SBSP', 'origem_SBSV', 'destino_SBBR',
       'destino_SBCF', 'destino_SBCT', 'destino_SBFL', 'destino_SBGL',
       'destino_SBGR', 'destino_SBKP', 'destino_SBPA', 'destino_SBRF',
       'destino_SBRJ', 'destino_SBSP', 'destino_SBSV', 'aero_esperas_SBBR',
       'aero_esperas_SBCF', 'aero_esperas_SBCT', 'aero_esperas_SBFL',
       'aero_esperas_SBGL', 'aero_esperas_SBGR', 'aero_esperas_SBKP',
       'aero_esperas_SBPA', 'aero_esperas_SBRF', 'aero_esperas_SBRJ',
       'aero_esperas_SBSP', 'aero_esperas_SBSV', 'aero_tcp_SBBR',
       'aero_tcp_SBCF', 'aero_tcp_SBCT', 'aero_tcp_SBFL', 'aero_tcp_SBGL',
       'aero_tcp_SBGR', 'aero_tcp_SBKP', 'aero_tcp_SBPA', 'aero_tcp_SBRF',
       'aero_tcp_SBRJ', 'aero_tcp_SBSP', 'aero_tcp_SBSV', 'aero_tcr_SBBR',
   

In [377]:
train_data.dt_dep, test_data.dt_dep = train_data.dt_dep.dt.hour, test_data.dt_dep.dt.hour

In [378]:
display(train_data.head())
display(test_data.head())

,flightid,esperas,troca,solution,dt_dep,origem_SBBR,origem_SBCF,origem_SBCT,origem_SBFL,origem_SBGL,...,aero_tcr_SBCT,aero_tcr_SBFL,aero_tcr_SBGL,aero_tcr_SBGR,aero_tcr_SBKP,aero_tcr_SBPA,aero_tcr_SBRF,aero_tcr_SBRJ,aero_tcr_SBSP,aero_tcr_SBSV
0,fcb2bf90345705318213ae1307c0f901,0,0,2463,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,c7c5c10716335b048f86d8c52fcba3f2,0,0,2152,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3162de7203a972f071d5a48e8f0f4828,0,0,4613,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8fcd243e9b9eebbe62ab145ce04b6ab5,0,0,2581,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,504a62621cd231d6ab67e674ce538cd3,0,0,5547,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,flightid,esperas,troca,dt_dep,origem_SBBR,origem_SBCF,origem_SBCT,origem_SBFL,origem_SBGL,origem_SBGR,...,aero_tcr_SBGL,aero_tcr_SBGR,aero_tcr_SBKP,aero_tcr_SBRF,aero_tcr_SBRJ,aero_tcr_SBSP,aero_tcr_SBBR,aero_tcr_SBFL,aero_tcr_SBPA,aero_tcr_SBSV
0,7945735584a3297121c4f5ae0de8ecd1,0,0,11,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,85cb958e8b9d583575cd9cb93e389226,0,0,8,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2d4e8cd3947525a12ff16af520beefb9,0,0,11,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,24cf317a1494a016a018d9f5cd183639,2,0,16,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,370babaf32dc82a6a64bf5b71a3434ed,0,0,22,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [379]:
features = train_data.columns
features = features.drop(['flightid', 'solution'])
target = 'solution'

X_train, X_test, y_train, y_test = train_test_split(train_data[features], train_data[target], test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'Linear Regression MSE: {mean_squared_error(y_test, y_pred)}')

# model2 = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
# model2.fit(X_train, y_train)
# y_pred = model2.predict(X_test)
# y_pred = np.round_(y_pred)
print(y_pred)
# print(f'Random Forest Regressor MSE: {mean_squared_error(y_test, y_pred)}')

Linear Regression MSE: 3127220.329384667
[3083.86328125 6321.10546875 3778.34765625 ... 3692.67578125 3692.67578125
 3040.        ]


In [380]:
# X_train, y_train = train_data[features], train_data[target]
# X_test = test_data[features]
# model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# y_pred = np.round_(y_pred)
# print(y_pred)

# output = pd.DataFrame({'ID': test_data.flightid, 'solution': y_pred})

# output.to_csv('data/submission.csv', index=False)

In [381]:
# metar_df.tail()

In [382]:
# metar_df.metar = metar_df.metar.apply(lambda x: x.replace('=', ' '))
# metar_sample = metar_df['metar'].sample(1).iloc[0]
# print(metar_sample)

# #metar_sample = 'METAR SBRF 152000Z 13005KT 9999 FEW020 SCT070 28/21 Q1011'

# report = Metar.Metar(metar_sample)
# print(report, '\n')

# print(f'Station_Id: {report.station_id}')
# print(f'Type: {report.type}')
# print(f'Time: {report.time}')
# print(f'Day: {report._day}')
# print(f'Temperature: {report.temp}')
# print(f'Dew Point: {report.dewpt}')
# print(f'Wind Direction: {report.wind_dir}')
# print(f'Visibility: {report.vis}')
# print(f'Wind Speed: {report.wind_speed}')
# print(f'Wind Gust: {report.wind_gust}')
# print(f'Pressure: {report.press}')
# print(f'Precipitation: {report.precip_1hr}')


In [383]:
metar_df.metar = metar_df.metar.apply(lambda x: x.replace('=', ''))

metar_df['temperature_real'] = None
metar_df['dew_point_real'] = None
metar_df['wind_direction_real'] = None
metar_df['visibility_real'] = None
metar_df['wind_speed_real'] = None
metar_df['pressure_real'] = None

count_missing = 0
count = 0
for i, report in enumerate(metar_df['metar']):
    count += 1
    try:
        report = Metar.Metar(report)
        metar_df.at[i, 'temperature_real'] = report.temp.value()
        metar_df.at[i, 'dew_point_real'] = report.dewpt.value()
        metar_df.at[i, 'wind_direction_real'] = report.wind_dir.value()
        metar_df.at[i, 'visibility_real'] = report.vis.value()
        metar_df.at[i, 'wind_speed_real'] = report.wind_speed.value()
        metar_df.at[i, 'pressure_real'] = report.press.value()
    except:
        count_missing += 1
        continue

print(count)
print(count_missing)
metar_df.sample(10)


106744
6434


,hora,metar,aero,temperature_real,dew_point_real,wind_direction_real,visibility_real,wind_speed_real,pressure_real
20939,2022-10-22 13:00:00,METAR SBCT 221300Z VRB02KT 9999 FEW030 21/13 Q...,SBCT,21.0,13.0,None,None,None,None
25779,2023-04-27 06:00:00,METAR SBCT 270600Z 27004KT CAVOK 15/14 Q1016,SBCT,15.0,14.0,270.0,10000.0,4.0,1016.0
28969,2022-09-18 01:00:00,METAR SBFL 180100Z 19001KT CAVOK 13/11 Q1024,SBFL,13.0,11.0,190.0,10000.0,1.0,1024.0
73377,2022-10-16 10:00:00,METAR SBRF 161000Z 21006KT 9999 SCT019 SCT060 ...,SBRF,25.0,21.0,210.0,10000.0,6.0,1014.0
97737,2023-04-29 05:10:00,SPECI SBSP 290510Z AUTO 12005KT 4700 -RA SCT00...,SBSP,16.0,16.0,120.0,4700.0,5.0,1020.0
99655,2022-07-29 09:00:00,METAR SBSV 290900Z 28002KT 9999 SCT020 SCT060 ...,SBSV,22.0,20.0,280.0,10000.0,2.0,1016.0
44701,2022-07-17 21:00:00,METAR SBGR 172100Z 16007KT 8000 OVC011 17/16 Q...,SBGR,17.0,16.0,160.0,8000.0,7.0,1023.0
59443,2023-03-12 11:00:00,METAR SBKP 121100Z 14003KT 9999 FEW020 22/20 Q...,SBKP,22.0,20.0,140.0,10000.0,3.0,1017.0
75009,2022-12-22 05:00:00,METAR SBRF 220500Z 30003KT 9999 BKN025 SCT050 ...,SBRF,25.0,22.0,300.0,10000.0,3.0,1010.0
11460,2022-09-29 12:00:00,METAR SBCF 291200Z 10009KT 9999 SCT025 BKN070 ...,SBCF,18.0,16.0,100.0,10000.0,9.0,1021.0


In [384]:
replace_equals_newline = lambda x: x.replace('=\n', ' ')
replace_metaf = lambda x: x.replace('METAF', 'METAR')

metaf_df.metaf = metaf_df.metaf.apply(replace_equals_newline)
metaf_df.metaf = metaf_df.metaf.apply(replace_metaf)
metaf_df.head()

,hora,metaf,aero
0,2022-08-05 00:00:00,METAR SBBR 050000Z 05006KT CAVOK 18/04 Q1014,SBBR
1,2022-08-05 01:00:00,METAR SBBR 050100Z 01006KT CAVOK 17/05 Q1016,SBBR
2,2022-08-05 02:00:00,METAR SBBR 050200Z 35008KT CAVOK 17/05 Q1014,SBBR
3,2022-08-05 03:00:00,METAR SBBR 050300Z 34007KT CAVOK 16/05 Q1015,SBBR
4,2022-08-05 04:00:00,METAR SBBR 050400Z 34008KT CAVOK 16/05 Q1015,SBBR


In [385]:
metaf_df.shape

(43938, 3)

In [386]:
metaf_df.metaf = metaf_df.metaf.apply(lambda x: x.replace('=', ''))

metaf_df['temperature_prev'] = None
metaf_df['dew_point_prev'] = None
metaf_df['wind_direction_prev'] = None
metaf_df['visibility_prev'] = None
metaf_df['wind_speed_prev'] = None
metaf_df['pressure_prev'] = None

count_missing = 0
count = 0
for i, report in enumerate(metaf_df['metaf']):
    count += 1
    try:
        report = Metar.Metar(report)
        metaf_df.at[i, 'temperature_prev'] = report.temp.value()
        metaf_df.at[i, 'dew_point_prev'] = report.dewpt.value()
        metaf_df.at[i, 'wind_direction_prev'] = report.wind_dir.value()
        metaf_df.at[i, 'visibility_prev'] = report.vis.value()
        metaf_df.at[i, 'wind_speed_prev'] = report.wind_speed.value()
        metaf_df.at[i, 'pressure_prev'] = report.press.value()
    except:
        count_missing += 1
        continue

In [387]:
metaf_df.head(30)

,hora,metaf,aero,temperature_prev,dew_point_prev,wind_direction_prev,visibility_prev,wind_speed_prev,pressure_prev
0,2022-08-05 00:00:00,METAR SBBR 050000Z 05006KT CAVOK 18/04 Q1014,SBBR,18.0,4.0,50.0,10000.0,6.0,1014.0
1,2022-08-05 01:00:00,METAR SBBR 050100Z 01006KT CAVOK 17/05 Q1016,SBBR,17.0,5.0,10.0,10000.0,6.0,1016.0
2,2022-08-05 02:00:00,METAR SBBR 050200Z 35008KT CAVOK 17/05 Q1014,SBBR,17.0,5.0,350.0,10000.0,8.0,1014.0
3,2022-08-05 03:00:00,METAR SBBR 050300Z 34007KT CAVOK 16/05 Q1015,SBBR,16.0,5.0,340.0,10000.0,7.0,1015.0
4,2022-08-05 04:00:00,METAR SBBR 050400Z 34008KT CAVOK 16/05 Q1015,SBBR,16.0,5.0,340.0,10000.0,8.0,1015.0
5,2022-08-05 05:00:00,METAR SBBR 050500Z 34007KT CAVOK 16/05 Q1015,SBBR,16.0,5.0,340.0,10000.0,7.0,1015.0
6,2022-08-05 06:00:00,METAR SBBR 050600Z 31005KT CAVOK 15/06 Q1014,SBBR,15.0,6.0,310.0,10000.0,5.0,1014.0
7,2022-08-05 07:00:00,METAR SBBR 050700Z 32004KT CAVOK 15/05 Q1015,SBBR,15.0,5.0,320.0,10000.0,4.0,1015.0
8,2022-08-05 08:00:00,METAR SBBR 050800Z 34006KT CAVOK 15/05 Q1012,SBBR,15.0,5.0,340.0,10000.0,6.0,1012.0
9,2022-08-05 09:00:00,METAR SBBR 050900Z 33007KT CAVOK 14/05 Q1013,SBBR,14.0,5.0,330.0,10000.0,7.0,1013.0


In [388]:
metaf_df.shape

(43938, 9)

In [389]:
metar_df['hora_ref'], metaf_df['hora_ref'] = metar_df['hora'].dt.strftime('%Y-%m-%d %H:%M:%S'), metaf_df['hora'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [390]:
metar_metaf = pd.merge(metar_df, metaf_df, how='left', on=['hora_ref'])
metar_metaf.head()

,hora_x,metar,aero_x,temperature_real,dew_point_real,wind_direction_real,visibility_real,wind_speed_real,pressure_real,hora_ref,hora_y,metaf,aero_y,temperature_prev,dew_point_prev,wind_direction_prev,visibility_prev,wind_speed_prev,pressure_prev
0,2022-06-01 00:00:00,METAR SBBR 010000Z 07002KT CAVOK 21/08 Q1018,SBBR,21.0,8.0,70.0,10000.0,2.0,1018.0,2022-06-01 00:00:00,2022-06-01 00:00:00,METAR SBGL 010000Z 25002KT 3000 BR OVC033...,SBGL,23.0,21.0,250.0,3000.0,2.0,1016.0
1,2022-06-01 00:00:00,METAR SBBR 010000Z 07002KT CAVOK 21/08 Q1018,SBBR,21.0,8.0,70.0,10000.0,2.0,1018.0,2022-06-01 00:00:00,2022-06-01 00:00:00,METAR SBGR 010000Z 12008KT 1000 BR OVC033...,SBGR,15.0,14.0,120.0,1000.0,8.0,1017.0
2,2022-06-01 00:00:00,METAR SBBR 010000Z 07002KT CAVOK 21/08 Q1018,SBBR,21.0,8.0,70.0,10000.0,2.0,1018.0,2022-06-01 00:00:00,2022-06-01 00:00:00,METAR SBKP 010000Z 12013KT 5000 BR BKN033...,SBKP,17.0,14.0,120.0,5000.0,13.0,1015.0
3,2022-06-01 00:00:00,METAR SBBR 010000Z 07002KT CAVOK 21/08 Q1018,SBBR,21.0,8.0,70.0,10000.0,2.0,1018.0,2022-06-01 00:00:00,2022-06-01 00:00:00,METAR SBSP 010000Z 13008KT 1000 BR OVC033...,SBSP,15.0,14.0,130.0,1000.0,8.0,1018.0
4,2022-06-01 01:00:00,METAR SBBR 010100Z 10002KT CAVOK 20/09 Q1019,SBBR,20.0,9.0,100.0,10000.0,2.0,1019.0,2022-06-01 01:00:00,2022-06-01 01:00:00,METAR SBGL 010100Z 02001KT 2000 BR OVC011 ...,SBGL,22.0,21.0,20.0,2000.0,1.0,1017.0
